In [8]:
from ipynb.fs.full.koselleck import *

[Koselleck] (13:52:08) Alles bereit 


In [12]:
# pd.Series(y for y,x in psgs).plot.density()

In [13]:
def gen_passages(words=None):
    words=get_valid_words() if words is None else words
    dfskips=get_skipgrams()
    return dfskips

In [14]:
def stream_skips(dfskips=None):
    dfskips=get_skipgrams().query('1720<=year<1900') if dfskips is None else dfskips
    for year,path in zip(dfskips.year, dfskips.path):
        with open(path) as f:
            for ln in f:
                yield (year,ln.strip())

In [15]:
def gen_passages_in_skip(dfskip,words=None):
    with get_veclib('psgs') as vl:
        words=get_valid_words() if not words else words
        words=set(words)
        for year,skipstr in stream_skips(dfskip):
            skiptoks=skipstr.split()
            matches=set(skiptoks)&words
            for match in matches:
                if not match in vl:
                    vl[match]={(year,skipstr)}
                else:
                    vl[match]=vl[match] | {(year,skipstr)}
        vl.commit()
        return pd.DataFrame()

In [16]:
# !rm ../db/db.koselleck.psgs.sqlite

In [17]:
for i,dfg in get_skipgrams().groupby('path'): pass

In [18]:
# get_skipgrams??

In [19]:
# gen_passages_in_skip(dfg)

In [20]:
# with get_veclib('psgs') as vl:
#     print(list(vl['language']))

In [21]:
# pmap_groups(
#     gen_passages_in_skip,
#     get_skipgrams().groupby('path'),
#     num_proc=1
# )

In [22]:
import re

In [23]:
def do_sents_word(obj):
    year,path,words,num_per_year=obj
    sents = defaultdict(set)
    o=[]
    with open(path) as f:
        for ln in f:
            skipstr=ln.strip()                
            skiptoks=skipstr.split()
            matches=set(skiptoks)&set(words)
            for match in matches:
                sents[match]|={(year,skipstr)}
            done=all(len(sents[w])>=num_per_year for w in words)
            if done: break

    for w,sl in sents.items():
        sl=random.sample(sl,num_per_year) if len(sl)>num_per_year else sl
        for y,s in sl:
            sent=s[0].upper()+s[1:]
            sent=re.sub(r"\b%s\b" % w,'**'+w.upper()+'**',sent)
            o+=[{'word':w, 'year':y, 'sent':sent}]
    return pd.DataFrame(o)

def sents_word(word_or_words,dfskips=None,num_per_year=10,num_proc=1):
    dfskips=get_skipgrams().query('1720<=year<1900') if dfskips is None else dfskips    

    words=tokenize_fast(word_or_words) if type(word_or_words)==str else word_or_words
    words_todo=[]
    o=[]
    with get_veclib('sents') as vl:
        for w in words:
            if w in vl and vl[w] is not None:
                o+=[vl[w]]
            else:
                words_todo+=[w]
    odf_old=pd.concat(o) if len(o) else pd.DataFrame()
    
    # get missing
    if len(words_todo):
        objs=[(year,path,words_todo,num_per_year) for year,path in zip(dfskips.year, dfskips.path)]
        random.shuffle(objs)
        o_new=pmap(do_sents_word, objs, num_proc=num_proc, desc='Gathering passages')
        odf_new=pd.concat(o_new).sort_values('year') if len(o_new) else pd.DataFrame()
        odf=odf_old.append(odf_new)
    else:
        odf=odf_old
    
    # cache
    with get_veclib('sents') as vl:
        for w,wdf in odf.groupby('word'): vl[w]=wdf
        vl.commit()
    return odf
    

In [9]:
odf2=sents_word('station',num_proc=4)
odf2

,word,year,sent
2,station,1720,Upon the whole matter the court unanimously agreed to the question for relieving the lait and then adjourn'd on the there was general court of the bank of england when the governor acquainted them that he presume none could be there had of late been divers meetings and conferences between the of this and the dire kors of the under the influence and interposition of some persons of the figure and **STATION** they had made no agreement yet with the se
0,station,1720,Bur may it please your grace we have many grounds of rejoycing one thing is matter of grief to us and asse ts us very sensibly the growth of popery in some parts of this land and that his majesty in his great goodness hath given us the kind intimation of his hopes that this grievance may be remedy'd by his royal care affords us most sensible pleasure the rather when we confider that the remarkable success his laudable are constantly attended with gives us great ground to hope that his endeavours to us of this grievance will be through the of god and the great encouragement we have from his calls us to do our part in our stations with diligence and application for obtaining these good ends preventing the growth of popery promoting true religion and piety and vice and immorality it please your grace should be wanting to my upon this occasion and fail in what i am perswaded is the mind of this should i omit to take notice that his majesty's choice of your grace to represent his royal person among us is most acceptable to us us considering with what constant zeal and unwearied application you have in the ere wherein providence has plac'd you and particularly i present high **STATION** contributed to advance the interest of this church
1,station,1720,Here committee was accordingly nam'd and the chairman wrote down the names of those which were given to him he was to have them read over to the company to know it they agreed to them which would have ended the when mr ai ers one of the present sheriffs of london pretented him by rising up and that he heard the honour of the king there brought in question and having the honour to be plac'd under his majesty in an **STATION** he thought this was tumultuous meeting and would read the proclamation against ri ot if they did not immediately disperse
3,station,1721,They had been but two days upon that **STATION** before they saw me and are both under the command of john roberts they carry'd me into there kept me five days and what of the cargo was not fit for their they threw overboard they took away most of my rigging and sails all my anchors blocks provisions powder small
2,station,1721,Slight is the subject but the praise not if heav'n assist and hear my call first for thy bees quiet **STATION** and lodge under covert of the wind
...,...,...,...
7,station,1899,How at zagazig when conducting one of the earliest tourist parties for whom he had provided lunch at the **STATION** refreshment room he found that he had left all his cash in courier bag in the railway carriage how on hurrying back for his money an officious railway servant an egyptian caught him in the act and arrested him as thief
8,station,1899,Mr cook altogether rejected any such proposals and was at once met with fierce reprisals the mountaineers attacked and burnt the railway **STATION** seized the rolling stock threw the carriages down the old crater of the volcano and cut the line
9,station,1899,Not much comfort to be found but the picturesque abounds my travelling was ani young pole painter who at the **STATION** on our way to the museum left me abruptly excuse me but i want to send off telegram to my mother
6,station,1899,We sailed from portsmouth on board the winchester my father's flagship at that time the brazils were included in the command of the cape **STATION** and we spent some time in rio janeiro where we were most hospitably entertained by the english minister mr hamilton


In [11]:
def highlight_psg(psg):
    return ' '.join(
        w if w!=w.upper() or w.startswith('(') else f'<b><u>{w.lower()}</u></b>'
        for w in psg.split()
    )

printm(highlight_psg('blah blah CULTURE blah blah (1711)'))

blah blah <b><u>culture</u></b> blah blah (1711)

In [51]:

def show_sents_word(word_or_df=None,n=5,n_per_dec=2,ybin=50,**attrs):
    odf=sents_word(word_or_df,**attrs) if type(word_or_df) in {str,list,tuple} else word_or_df
    odf['period']=[periodize_sattelzeit(y) if ybin==None else periodize(y,ybin) for y in odf.year]
    odf['decade']=[periodize(y,10) for y in odf.year]
    odf=odf.groupby('decade').sample(n=n_per_dec,replace=True)
    odf=odf.groupby('period').sample(n=n,replace=True)
    odf['sent']=[f'{s} ({y})' for s,y in zip(odf.sent, odf.year)]
    odf=pd.concat(dfg.assign(i=list(range(len(dfg)))) for i,dfg in odf.groupby(['word','period']))
    odf=odf.pivot('i','period','sent').reset_index().drop('i',1)
    for col in odf: odf[col]=sorted(odf[col], key=lambda x: str(x).split()[-1])
    return odf

In [50]:
printm(show_sents_word('value').to_markdown())

|    | 1700-1750                                                                                                                                                                                                                                                                                                           | 1750-1800                                                                                                                                                                                                                                                                                                                                                   | 1800-1850                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | 1850-1900                                                                                                                                                                                                                                                                                                                                                                                                |
|---:|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | The on exchanging every such bill take in the bill fo exchang'd and stand of the fame in trust for the company and hall be answerable to them for the principal and interest to grow due thereon during the time they shall be in the hands of the or the **VALUE** thereof in like ais (1720)                      | Large number of fugitive political trails which had their **VALUE** when the incidents were actually on the great of business came from his pen the periodical paper called i he champion owed its chief sup (1764)                                                                                                                                         | By ross useful legal information to purchasers and possessors of estates houses annuities **VALUE** and security with the best modes of investing money with every necessary table of calculations particularly useful to solicitors conveyancers auctioneers by (1812)                                                                                                                                                                                                                                                                                                                                                                                                                                                              | For six years it stood out against endorsing it until quite recently forceed by the pressure from without it to countenance it having done this we have to inquire what is the **VALUE** of their endorsement of (1854)                                                                                                                                                                                  |
|  1 | Treasure and tit to be communicated to every one left too much with it might have discovered too milch of its **VALUE** and if the has but entailed his fa of dreaming no matter for that particular piece his heirs can never want as many of the like opportunities of serving their country as they quest (1728) | It bs there is not much in this but there not certain ec the to be acquired call be only in society ho kt so not the **VALUE** of friend (1772)                                                                                                                                                                                                             | Is i and there are of it in truth are ait st for more than two ries the of england ill te lures il spite of ll their articles uni of faith and hae been by those of tile ill i nonconformity i **VALUE** but whose i le bv i of til ere i (1825)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | To this must that the **VALUE** of the current coins specially of the piaster piece was of fluctuating nature it anus then the practice of the district governors to lower the **VALUE** by proclamation when the taxes bael to be collected and to raise after they had been collected (1872)                                                                                                           |
|  2 | Nothing bult the spirit of gaming can for the the gives them after such impositions cistern diamond necklace tea equipage or silver commonly second hand and much their real **VALUE** are to put off as trinkets as the undertaker thinks fir (1738)                                                               | To gratify our pride see dare the tide th' inclemencies of either zone inventive india's proudest boast the elegance of china's coast what distant climates **VALUE** most pow'rful aid are all our own (1775)                                                                                                                                              | We ha ve shown that error of of land lies in the calke of **VALUE** of his property le took the artificial to be the natural and general price (1832)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | From realising stock and other investments he proceeded to close his banking accounts to convert the land purchased by his father at into money though every day at that period increased its **VALUE** to sell down to the last grape or peach the yearly products of his vinery and orchard houses (1887)                                                                                              |
|  3 | But will virtuous lady fay he can't fet due **VALUE** on her honour without the oi ce in the immodest scenes of an adulteress hu and's bed (1743)                                                                                                                                                                   | Ment the high bailiff of though solemnly called upon refused to make any return of citizens to serve in parliament for the said city that your impressed with high sense of the **VALUE** of that branch of the legislature which they have been taught to confider as the natural guardian of the rights of the people from whom it derives its pow (1785) | Some of these of the inferior grade we have already met with on the eastern extremity of the border of ulster but on the boundary of that province towards the west the accumulation of misery was originally much greater and here in sligo and northern mayo particularly the consequences of two centuries of degradation and hardship exhibit themselves in the whole physical condition of the people affecting not only the features but the frame and giving such an example of human deterioration from known causes as almost compensates by its **VALUE** to future ares for the suffering and debasement that past generations have endured in perfecting its appalling lesson in the persons of their descendants (1836) | They kindled fire within it for it was perforated and hollow and every man came in after the **VALUE** of his offering or he who came in with fowl entered the first chancel he who came in with sheep entered the second with lamb the third with calf the fourth with heifer the fifth with an ox the sixth but he who came in with his own begotten child him they caused to enter the seventh (1890) |
|  4 | The prisoners that were pvt on board the walpole and pelham compute that the **VALUE** of each of the even ship taken by the centurion aa above is on an average worth each (1747)                                                                                                                                  | Corn was about half its present **VALUE** other commodities much cheaper bishop fleetwood has determined from mod accurate of every that in this reign was equivalent to or (1789)                                                                                                                                                                          | Few sub appertaining to milit matt of late occupied so large share of public attention as the question of purchase in the army while all are willing to acknowledge the hardship of the case where old and meritorious services are passed over for the mere claim of money unsupported by qualification vet on the other hand no one can underrate the **VALUE** of connecting the higher and wealthier classes with service whose whole tone they have tended to elevate and who under the system promotion might not be found within its ranks (1842)                                                                                                                                                                             | He smooth clawed frog has lately begun to breed here thing before unknown so that his rarity and **VALUE** are in danger of depreciation but such is his inordinate conceit of himself that i am convinced he will always begin the bid ding with eight (1894)                                                                                                                                           |

In [27]:
show_sents_word(odf2)

,word,year,sent
2,station,1720,Upon the whole matter the court unanimously agreed to the question for relieving the lait and then adjourn'd on the there was general court of the bank of england when the governor acquainted them that he presume none could be there had of late been divers meetings and conferences between the of this and the dire kors of the under the influence and interposition of some persons of the figure and **STATION** they had made no agreement yet with the se
0,station,1720,Bur may it please your grace we have many grounds of rejoycing one thing is matter of grief to us and asse ts us very sensibly the growth of popery in some parts of this land and that his majesty in his great goodness hath given us the kind intimation of his hopes that this grievance may be remedy'd by his royal care affords us most sensible pleasure the rather when we confider that the remarkable success his laudable are constantly attended with gives us great ground to hope that his endeavours to us of this grievance will be through the of god and the great encouragement we have from his calls us to do our part in our stations with diligence and application for obtaining these good ends preventing the growth of popery promoting true religion and piety and vice and immorality it please your grace should be wanting to my upon this occasion and fail in what i am perswaded is the mind of this should i omit to take notice that his majesty's choice of your grace to represent his royal person among us is most acceptable to us us considering with what constant zeal and unwearied application you have in the ere wherein providence has plac'd you and particularly i present high **STATION** contributed to advance the interest of this church
1,station,1720,Here committee was accordingly nam'd and the chairman wrote down the names of those which were given to him he was to have them read over to the company to know it they agreed to them which would have ended the when mr ai ers one of the present sheriffs of london pretented him by rising up and that he heard the honour of the king there brought in question and having the honour to be plac'd under his majesty in an **STATION** he thought this was tumultuous meeting and would read the proclamation against ri ot if they did not immediately disperse
3,station,1721,They had been but two days upon that **STATION** before they saw me and are both under the command of john roberts they carry'd me into there kept me five days and what of the cargo was not fit for their they threw overboard they took away most of my rigging and sails all my anchors blocks provisions powder small
2,station,1721,Slight is the subject but the praise not if heav'n assist and hear my call first for thy bees quiet **STATION** and lodge under covert of the wind
...,...,...,...
7,station,1899,How at zagazig when conducting one of the earliest tourist parties for whom he had provided lunch at the **STATION** refreshment room he found that he had left all his cash in courier bag in the railway carriage how on hurrying back for his money an officious railway servant an egyptian caught him in the act and arrested him as thief
8,station,1899,Mr cook altogether rejected any such proposals and was at once met with fierce reprisals the mountaineers attacked and burnt the railway **STATION** seized the rolling stock threw the carriages down the old crater of the volcano and cut the line
9,station,1899,Not much comfort to be found but the picturesque abounds my travelling was ani young pole painter who at the **STATION** on our way to the museum left me abruptly excuse me but i want to send off telegram to my mother
6,station,1899,We sailed from portsmouth on board the winchester my father's flagship at that time the brazils were included in the command of the cape **STATION** and we spent some time in rio janeiro where we were most hospitably entertained by the english minister mr hamilton


period,1700-1750,1750-1800,1800-1850,1850-1900
0,Thei his said gave pu lick order fur tht latter jest the day in familiar for my part i have spent the greater half of my life in this **STATION** of holy service i thank god not not unprofitably but there is no one thing whereof i repent fo much as not to more hours in this publick exercise of catechism in regard whereof i could quarrel with my very sermons (1728),Amongst the reft there an old man who lived very decently upon narrow income and took great pains in weeding up his children he was baking his bread in little cakes when the cardinal came on which he ran up flairs to change his cloaths but he made him come down immediately your dress and your business said he very well become your **STATION** take care that your bread don't burn we can discourse together while you are turning it (1761),It was here that the celebrated lord william howard better known as bold or belted will howard man lord william says mr howard in his memorials of the howard family is in the lay of the last minstrel called by sir walter scott belted will howard meaning i apprehend that he was in the habit of wearing the baldrick or broad belt which was formerly worn as distinguishing badge by persons of high **STATION** (1839),February at three o'clock by of the london academy of music assisted by miss adam and signor guido violin messrs wolff co's agents will be happy to send tickets to organists and amateurs desirous of attending on application to them at new bond street organ recitals at lancaster hall lancaster road near notting hill **STATION** (1880)
1,Which he thought nobody would but that the matter to be considered was whether signor nuncio at the court of lisbon having been recall'd by the pope his sovereign ho is not to give an account of his reasons to any body in the world and having not obey'd his orders but on the contrary continued in his **STATION** and as nuncio since his being recall'd is not guilt of downright disobedience and rebellion amounting very near to high treason his sovereign (1728),And in hearty for the peace and welfare of every christian is submitted to your own most attentive consideration nd it is submitted in this public manner that it at the fame draw the attention of on your who their **STATION** and repute both for learning and piety fairly be pres med to have moss weigh with man of modesty and candour (1781),It is to rebut most directly any mich insinuation that i now obtrude self upon your notice could it for moment be intended that dreadful personal animosities were felt by he exalted personage who now lills the **STATION** of father to his people (1812),While at those famous health resorts the names of which are to be read at every railway **STATION** and in the advertisement sheets of every newspaper hund eds and thousands are coming back from weakness to strength there are hundreds and thousands still who are discharged from our great metropolitan hospitals to creep to rooms in dim close courts and alloys where all tending care that can be given them must be snatched from the hours of labour necessary to buy medicine and food (1874)
2,This is the first lady that has been nam'd to serve her majesty in that **STATION** ance her coming to en viand richard lomax gent is made ensign of in the hon (1731),To introduce equality into heaven was the and the heinous fin of those rebellious angels which were cast down into the abyss the proud and discontented lucifer not pleased with the **STATION** which god had allotted him drew away the third part of heaven after him and took up arms against the almighty (1795),I to judge of the morals characters of men by **STATION** rather than by profession and with genuine spirit of an englishman i sought for virtue within precincts of court (1813),I saw only one instance of this we were rejoining the main road after having paid visit to central about six in the evening we had reached new **STATION** (1897)
3,When any species of c